In [95]:
import itertools
import pandas as pd
import sqlalchemy as sqa
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input

DB = f'postgres://localhost/{"poker"}'
ENGINE = sqa.create_engine(DB)

max_features = 53
max_len = 7
vector_size = 4

In [105]:
df = pd.read_sql_table("results", con=ENGINE)

In [106]:
y = df[['action', 'reward']]

In [97]:
cards = np.array(df[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[0])
for i in range(1, df.shape[0]):
    cards = np.vstack((cards, np.array(df[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[i])))
cards

array([[ 25.,  22., 999., ..., 999., 999., 999.],
       [ 44.,  51., 999., ..., 999., 999., 999.],
       [ 31.,  35., 999., ..., 999., 999., 999.],
       ...,
       [ 49.,  34., 999., ..., 999., 999., 999.],
       [ 31.,  46., 999., ..., 999., 999., 999.],
       [  8.,   1., 999., ..., 999., 999., 999.]])

In [99]:
cards.shape

(244, 7)

Create a model:

In [102]:
# Card embeddings
card_input = Input(shape=cards.shape)
card_output = layers.Embedding(max_features, vector_size, input_length=max_len)(card_input)

# Other information


In [93]:
model = models.Sequential()

model.add(layers.Embedding(
    max_features, vector_size, input_length=max_len
    # Number of distinct features, size of the word vector, input size
                          ))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='softmax'))
model.summary()

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 4)              212       
_________________________________________________________________
flatten_1 (Flatten)          (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 29        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [103]:
df.columns

Index(['index', 'action', 'active_0', 'active_1', 'active_2', 'active_3',
       'active_4', 'bid', 'bid_0', 'bid_1', 'bid_2', 'bid_3', 'bid_4',
       'community1', 'community2', 'community3', 'community4', 'community5',
       'hand1', 'hand2', 'player', 'position', 'round', 'reward'],
      dtype='object')

In [ ]:
rest = df[['']]

In [14]:
df.head()

,index,action,active_0,active_1,active_2,active_3,active_4,bid,bid_0,bid_1,...,community2,community3,community4,community5,hand1,hand2,player,position,round,reward
0,0,4.0,1.0,1.0,1.0,1.0,1.0,200.0,100.0,200.0,...,999.0,999.0,999.0,999.0,25.0,22.0,2.0,2.0,1.0,-400.0
1,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0,200.0,...,999.0,999.0,999.0,999.0,44.0,51.0,3.0,3.0,1.0,0.0
2,2,4.0,1.0,1.0,1.0,0.0,1.0,200.0,100.0,200.0,...,999.0,999.0,999.0,999.0,31.0,35.0,4.0,4.0,1.0,-1000.0
3,3,5.0,1.0,1.0,1.0,0.0,1.0,400.0,100.0,200.0,...,999.0,999.0,999.0,999.0,11.0,42.0,5.0,5.0,1.0,-1600.0
4,4,4.0,1.0,1.0,0.0,1.0,1.0,400.0,200.0,200.0,...,999.0,999.0,999.0,999.0,0.0,8.0,0.0,0.0,1.0,-2000.0


In [86]:
class Agent:
    '''
    The class Agent takes the following parameters as input:
    
    and uses them to decide on his action in a specified game of poker
    '''
    
    def __init__(self, df):
        self.dataframe = df
        self.input_card_embedding = ""
        self.rewards = df[['action', 'reward']]
    
    def create_embedding_input(self):
        '''
        The function create_embedding_input creates the input for the embedding of the card vectors
        '''
        cards = np.array(self.dataframe[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[0])
        for i in range(1, df.shape[0]):
            cards = np.vstack((cards, np.array(self.dataframe[['hand1', 'hand2', 'community1', 'community2', 'community3', 'community4', 'community5']].loc[i])))
        
        self.input_card_embedding = cards

In [87]:
agent = Agent(df)

In [88]:
agent.create_embedding_input()

In [89]:
agent.input_card_embedding

array([[ 25.,  22., 999., ..., 999., 999., 999.],
       [ 44.,  51., 999., ..., 999., 999., 999.],
       [ 31.,  35., 999., ..., 999., 999., 999.],
       ...,
       [ 49.,  34., 999., ..., 999., 999., 999.],
       [ 31.,  46., 999., ..., 999., 999., 999.],
       [  8.,   1., 999., ..., 999., 999., 999.]])

In [4]:
df.hand1[0], df.hand2[1]

(25.0, 51.0)

Let us look at the community cards:

In [8]:
for i in range(df.shape[0]):
    if len(df.community[i]) > 2:
        print(df.community[i])

{"(q,c)","(5,d)","(7,c)"}
{"(q,c)","(5,d)","(7,c)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(q,c)","(5,d)","(7,c)","(j,d)"}
{"(3,d)","(t,h)","(a,c)"}
{"(3,d)","(t,h)","(a,c)"}
{"(3,d)","(t,h)","(a,c)","(q,c)"}
{"(3,d)","(t,h)","(a,c)","(q,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(7,s)","(9,c)","(k,d)"}
{"(3,h)","(2,c)","(4,h)"}
{"(3,h)","(2,c)","(4,h)"}
{"(3,h)","(2,c)","(4,h)","(a,c)"}
{"(3,h)","(2,c)","(4,h)","(a,c)"}
{"(k,d)","(2,c)","(5,d)"}
{"(k,d)","(2,c)","(5